# Capstone Project Data Science | Site recommendation and competitor analysis for Latin Restaurant

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)


##Introduction - Business Problem
Restaurants and Culinary habits have always been one of the favorite ways to socialize and meet each other, have interesting conversation beside closing business contract. Knowing these habits, Restaurant income represent a good share of every country GDP (Gross domestic product). 

One the most complex decision for every restaurant it’s finding a perfect location for introducing a new option for neighborhood and surrounding area at a selected city.

My project it’s a about determining **best restaurant location and segmenting** competitors in Bronx city to find **low Latin restaurant desensity** by means of data science **identify best location** for opening a new restautant.

Bronx city has an increasing total immigrant from Latin America one of the first things people do when immigrating to a new country or city is to establish communities and open restaurants to feed them.

**The Bronx is the only New York borough with a Hispanic majority**. At the 2010 Census, 53.5% of Bronx's population was of Hispanic, Latino, or Spanish origin (they may be of any race). At the 2009 American Community Survey, Puerto Ricans represented 23.2% of the borough's population, Mexicans made 5.2%.In 2005, more than 200,000 Dominicans called the Bronx home.

###My target Audience:

Restaurant groups, retail chains, and other enterprises can use this work and use POI data to help solve problems such as site selection, competitive analysis, and much more.


#Data section
###We will be using basically two data sources for our analysis 

The restaurant data used in this study was pulled from **Foursquare’s Places Database** - a comprehensive dataset spanning 190+ countries and 50 territories. The data set is updated continuously and published daily as our world is constantly evolving. 
The Foursquare Places Database has a rich set of attributes you could use to understand the world around you.  This analysis only needed a few of them:  the restaurant’s location, the restaurant’s categorization, and information telling us if the restaurants were part of a national chain.  



### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. 

Let's first find the latitude & longitude of Bronx borough center, using specific, well known address and Google Maps geocoding API.

###Fisrt let's intall all libraries will be need for downlaod data and exploration

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy  
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


###Download and Explore Dataset
In order to **segement the neighborhoods** and explore them, we will essentially need a dataset that contains the Bronx boroughs and the neighborhoods as well as the the latitude and logitude coordinates of each neighborhood.

In [2]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')


Data downloaded!


In [3]:
#Load NewYork Data
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
newyork_data

{'bbox': [-74.2492599487305,
  40.5033187866211,
  -73.7061614990234,
  40.9105606079102],
 'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [-73.84720052054902,
     40.89470517661],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.1',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661],
    'borough': 'Bronx',
    'name': 'Wakefield',
    'stacked': 1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.82993910812398, 40.87429419303012],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.2',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.874294193

###Tranform the data into a pandas dataframe

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

#Confirm Comlumns
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [6]:
neighborhoods_data = newyork_data['features']

In [7]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

####let's loop through the data and fill the dataframe.

In [8]:
#Define Looping
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
  

In [9]:
#First 5 rows
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


###let's slice the original dataframe and create a new dataframe of the Bronx data.

In [10]:
bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

# Let's get the geographical coordinates of Bronx.
address = 'Bronx, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bronx are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bronx are 40.8466508, -73.8785937.


###Visualize Bronx the neighborhoods in it.

In [11]:
# create map of Bronx using latitude and longitude values
map_bronx = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(bronx_data['Latitude'], bronx_data['Longitude'], bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.8,
        parse_html=False).add_to(map_bronx)  
    
map_bronx

###Foursquare API to explore the neighborhoods and segment them

In [12]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'K5KBSAXPPKQZTRNZ55DN40DW5GC2TKMYE1010M45JRV1PY3B' # your Foursquare ID
CLIENT_SECRET = 'G4OPRLDBDU1PF1OUYDZOBKPES4Y1DCZ0WB5HCCYAOS5ZUZU5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K5KBSAXPPKQZTRNZ55DN40DW5GC2TKMYE1010M45JRV1PY3B
CLIENT_SECRET:G4OPRLDBDU1PF1OUYDZOBKPES4Y1DCZ0WB5HCCYAOS5ZUZU5


###Explore Neighborhoods in Bronx and venues category

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

###Let's create a neew dataframe with Bronx Venues

In [14]:

bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude']
                                  )

print(bronx_venues.shape)
bronx_venues.head()

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights
(3258, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Caribbean Restaurant
2,Wakefield,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Caribbean Restaurant
3,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant
4,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop


####How many venue categories do we have

In [15]:
bronx_venues.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
ATM,3,3,3,3,3,3
Accessories Store,3,3,3,3,3,3
African Restaurant,3,3,3,3,3,3
American Restaurant,32,32,32,32,32,32
Arcade,3,3,3,3,3,3
Arepa Restaurant,3,3,3,3,3,3
Art Gallery,9,9,9,9,9,9
Art Museum,3,3,3,3,3,3
Arts & Crafts Store,1,1,1,1,1,1


#### Now let's make a New Dataframe only Restaurant Category

In [16]:
bronx_venues_rest =bronx_venues[bronx_venues['Venue Category'].str.contains('Restaurant')]
bronx_venues_rest.shape


(607, 7)

In [17]:
bronx_venues_rest.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Caribbean Restaurant
2,Wakefield,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Caribbean Restaurant
3,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant
10,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant,40.898083,-73.850259,Caribbean Restaurant
19,Wakefield,40.894705,-73.847201,McDonald's,40.902645,-73.849485,Fast Food Restaurant


####Let's now see all the collected restaurants in our area of interest on map, and see the low reastaurant density area.

#####Let's how many unique sub restaurante categories we have in Bronx

In [18]:
print('There are {} uniques categories.'.format(len(bronx_venues_rest['Venue Category'].unique())))

There are 33 uniques categories.


#####Let's plot restaurant categories venues for exploring the most densed region in Bronx Borough

In [19]:
# create map of Bronx using latitude and longitude values
map_bronx_venues_restaurant = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(bronx_venues_rest['Venue Latitude'], bronx_venues_rest['Venue Longitude'], bronx_venues_rest['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color = 'blue' ,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_bronx_venues_restaurant)  
    
map_bronx_venues_restaurant

#####This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Latin restaurant!

## Methodology <a name="methodology"></a>


In this project we will make an effort on detecting areas of Bronxn that have low restaurant density, particularly those with low number of **Latin and Caibbean Restaurant**.

In first step we have collected the required data: **location and type (category)** of every restaurant within Bronx borough. We have also identified  **venues restaurants** (according to Foursquare categorization).

Second step in our analysis will be **segmenting areas based on restaurant density across different areas of Bronx** - we will use K-Means clustering Machine learning methods to identify bourough cluster and focus our attention in those cluster with lowest Restaurant density.

In third and final step we will focus on most promising areas and within those create clusters of locations that meet some basic requirements established in discussion with stakeholders: we will take into consideration locations with **lowest number of 1rst common veneu of caribbean restaurant**. We will present map of all such locations but also create clusters (using k-means clustering) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for later deeper analysis

#Analysis

In [20]:
# one hot encoding
bronx_onehot = pd.get_dummies(bronx_venues_rest[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bronx_onehot['Neighborhood'] = bronx_venues_rest['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bronx_onehot.columns[-1]] + list(bronx_onehot.columns[:-1])
bronx_onehot = bronx_onehot[fixed_columns]

bronx_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Eastern European Restaurant,Empanada Restaurant,Fast Food Restaurant,French Restaurant,Greek Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Peruvian Restaurant,Puerto Rican Restaurant,Restaurant,Seafood Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
1,Wakefield,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,Wakefield,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19,Wakefield,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


##### How many rows for our new data frame


In [21]:
bronx_onehot.shape

(607, 34)

##### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [22]:
bronx_grouped = bronx_onehot.groupby('Neighborhood').mean().reset_index()
bronx_grouped

,Neighborhood,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Eastern European Restaurant,Empanada Restaurant,Fast Food Restaurant,French Restaurant,Greek Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Peruvian Restaurant,Puerto Rican Restaurant,Restaurant,Seafood Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Allerton,0.000000,0.083333,0.000000,0.000000,0.250000,0.250000,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Baychester,0.000000,0.083333,0.000000,0.000000,0.083333,0.166667,0.000000,0.000000,0.000000,0.00,0.083333,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.166667,0.0,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bedford Park,0.000000,0.000000,0.000000,0.000000,0.090909,0.181818,0.000000,0.000000,0.000000,0.00,0.090909,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.272727,0.0,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.181818,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Belmont,0.038462,0.038462,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.038462,0.00,0.038462,0.000000,0.000000,0.000000,0.000000,0.576923,0.000000,0.115385,0.038462,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Bronxdale,0.000000,0.000000,0.066667,0.000000,0.000000,0.266667,0.000000,0.000000,0.066667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.066667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000
5,Castle Hill,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.250000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,City Island,0.000000,0.133333,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.00,0.000000,0.066667,0.000000,0.000000,0.000000,0.200000,0.066667,0.066667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Claremont Village,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.00,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Co-op City,0.000000,0.090909,0.000000,0.000000,0.181818,0.090909,0.000000,0.000000,0.000000,0.00,0.181818,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.181818,0.0,0.000000,0.000000,0.000000,0.090909,0.090909,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Concourse,0.000000,0.076923,0.000000,0.076923,0.076923,0.153846,0.000000,0.000000,0.000000,0.00,0.153846,0.000000,0.000000,0.076923,0.000000,0.076923,0.000000,0.076923,0.076923,0.0,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


#####Let's print each neighborhood along with the top 10 most common venues

In [23]:
num_top_venues = 10

for hood in bronx_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bronx_grouped[bronx_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                             venue  freq
0             Caribbean Restaurant  0.25
1               Chinese Restaurant  0.25
2             Fast Food Restaurant  0.25
3               Mexican Restaurant  0.17
4              American Restaurant  0.08
5  Southern / Soul Food Restaurant  0.00
6          New American Restaurant  0.00
7              Peruvian Restaurant  0.00
8          Puerto Rican Restaurant  0.00
9                       Restaurant  0.00


----Baychester----
                  venue  freq
0    Spanish Restaurant  0.17
1    Chinese Restaurant  0.17
2    Italian Restaurant  0.17
3    Mexican Restaurant  0.17
4  Caribbean Restaurant  0.08
5  Fast Food Restaurant  0.08
6    Seafood Restaurant  0.08
7   American Restaurant  0.08
8      Theme Restaurant  0.00
9       Thai Restaurant  0.00


----Bedford Park----
                           venue  freq
0             Mexican Restaurant  0.27
1             Chinese Restaurant  0.18
2             Spanish Restaurant  0.18
3 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


                     venue  freq
0      American Restaurant  0.22
1       Italian Restaurant  0.22
2       Chinese Restaurant  0.22
3               Restaurant  0.11
4         Sushi Restaurant  0.11
5       Mexican Restaurant  0.11
6       African Restaurant  0.00
7  New American Restaurant  0.00
8      Peruvian Restaurant  0.00
9  Puerto Rican Restaurant  0.00


----Norwood----
                       venue  freq
0         Mexican Restaurant  0.25
1       Fast Food Restaurant  0.19
2         Spanish Restaurant  0.19
3       Caribbean Restaurant  0.12
4         Chinese Restaurant  0.12
5                 Restaurant  0.06
6  Latin American Restaurant  0.06
7           Theme Restaurant  0.00
8            Thai Restaurant  0.00
9           Tapas Restaurant  0.00


----Olinville----
                           venue  freq
0           Fast Food Restaurant  0.31
1           Caribbean Restaurant  0.23
2             Chinese Restaurant  0.23
3             Mexican Restaurant  0.08
4  Vegetarian / Veg

####Create dataframe with top 10 venues and category frecuency

In [24]:
# Create a function to sort venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#####Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bronx_grouped['Neighborhood']

for ind in np.arange(bronx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bronx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Caribbean Restaurant,Chinese Restaurant,Fast Food Restaurant,Mexican Restaurant,American Restaurant,Vietnamese Restaurant,Empanada Restaurant,Indian Restaurant,Greek Restaurant,French Restaurant
1,Baychester,Italian Restaurant,Spanish Restaurant,Chinese Restaurant,Mexican Restaurant,American Restaurant,Caribbean Restaurant,Seafood Restaurant,Fast Food Restaurant,Empanada Restaurant,Greek Restaurant
2,Bedford Park,Mexican Restaurant,Spanish Restaurant,Chinese Restaurant,Italian Restaurant,Caribbean Restaurant,Fast Food Restaurant,New American Restaurant,Empanada Restaurant,Greek Restaurant,French Restaurant
3,Belmont,Italian Restaurant,Latin American Restaurant,Spanish Restaurant,Mexican Restaurant,Fast Food Restaurant,Eastern European Restaurant,African Restaurant,Caribbean Restaurant,American Restaurant,Chinese Restaurant
4,Bronxdale,Italian Restaurant,Chinese Restaurant,Spanish Restaurant,Eastern European Restaurant,Arepa Restaurant,Sushi Restaurant,Mexican Restaurant,Greek Restaurant,French Restaurant,Fast Food Restaurant


#Cluster Neighborhoods based on most common restaurant category

#####Run k-means to cluster the neighborhood into 4 clusters.

In [26]:
# set number of clusters
kclusters = 4

bronx_grouped_clustering = bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 0, 0, 2, 0, 1, 2, 2], dtype=int32)

#####Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bronx_merged = bronx_data

# merge bronx_grouped with bronx_data to add latitude/longitude for each neighborhood
bronx_merged = bronx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how='inner')

bronx_merged.head()


,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,1,Caribbean Restaurant,Fast Food Restaurant,Chinese Restaurant,Spanish Restaurant,Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant
1,Bronx,Co-op City,40.874294,-73.829939,2,Fast Food Restaurant,Caribbean Restaurant,Mexican Restaurant,Restaurant,American Restaurant,Spanish Restaurant,Chinese Restaurant,Seafood Restaurant,Eastern European Restaurant,French Restaurant
2,Bronx,Eastchester,40.887556,-73.827806,1,Caribbean Restaurant,Fast Food Restaurant,Asian Restaurant,Seafood Restaurant,Chinese Restaurant,Eastern European Restaurant,Indian Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant
3,Bronx,Fieldston,40.895437,-73.905643,2,Mexican Restaurant,Restaurant,Sushi Restaurant,Fast Food Restaurant,Chinese Restaurant,Greek Restaurant,Tapas Restaurant,Asian Restaurant,Caribbean Restaurant,Indian Restaurant
4,Bronx,Riverdale,40.890834,-73.912585,2,Japanese Restaurant,Mexican Restaurant,Tapas Restaurant,Italian Restaurant,Restaurant,Thai Restaurant,Greek Restaurant,Arepa Restaurant,Asian Restaurant,Caribbean Restaurant


#####Visualize restaurant cluster and neighborhood

In [28]:
# create map
bronx_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bronx_merged['Latitude'], bronx_merged['Longitude'], bronx_merged['Neighborhood'], bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=8,
        popup=label,
        color= 'Blue',#rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(bronx_map_clusters)
       
bronx_map_clusters

#####Now, we are going deeper each cluster and determine the discriminating venue categories that distinguish each cluster.

###Cluster One

In [29]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 0, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Woodlawn,American Restaurant,Indian Restaurant,Caribbean Restaurant,Chinese Restaurant,Italian Restaurant,Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Empanada Restaurant,Greek Restaurant
10,Pelham Parkway,Chinese Restaurant,Italian Restaurant,Eastern European Restaurant,Sushi Restaurant,Vietnamese Restaurant,Indian Restaurant,Greek Restaurant,French Restaurant,Fast Food Restaurant,Empanada Restaurant
11,City Island,Seafood Restaurant,Italian Restaurant,American Restaurant,Chinese Restaurant,Latin American Restaurant,French Restaurant,Japanese Restaurant,Spanish Restaurant,Caribbean Restaurant,Indian Restaurant
15,Fordham,Italian Restaurant,Spanish Restaurant,Latin American Restaurant,Vietnamese Restaurant,Mexican Restaurant,American Restaurant,Caribbean Restaurant,Chinese Restaurant,Fast Food Restaurant,African Restaurant
27,Throgs Neck,Italian Restaurant,Japanese Restaurant,American Restaurant,Asian Restaurant,Arepa Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Eastern European Restaurant
28,Country Club,Italian Restaurant,Mexican Restaurant,Fast Food Restaurant,American Restaurant,Thai Restaurant,Sushi Restaurant,Asian Restaurant,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant
31,Van Nest,Italian Restaurant,Spanish Restaurant,American Restaurant,Thai Restaurant,Arepa Restaurant,Caribbean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant,Empanada Restaurant
32,Morris Park,Italian Restaurant,Spanish Restaurant,Fast Food Restaurant,American Restaurant,Mexican Restaurant,Restaurant,Arepa Restaurant,Asian Restaurant,Sushi Restaurant,Latin American Restaurant
33,Belmont,Italian Restaurant,Latin American Restaurant,Spanish Restaurant,Mexican Restaurant,Fast Food Restaurant,Eastern European Restaurant,African Restaurant,Caribbean Restaurant,American Restaurant,Chinese Restaurant
35,North Riverdale,Italian Restaurant,American Restaurant,Chinese Restaurant,Sushi Restaurant,Mexican Restaurant,Restaurant,Caribbean Restaurant,Asian Restaurant,Comfort Food Restaurant,Indian Restaurant


###Cluster Two

In [30]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 1, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Caribbean Restaurant,Fast Food Restaurant,Chinese Restaurant,Spanish Restaurant,Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant
2,Eastchester,Caribbean Restaurant,Fast Food Restaurant,Asian Restaurant,Seafood Restaurant,Chinese Restaurant,Eastern European Restaurant,Indian Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant
8,Williamsbridge,Caribbean Restaurant,Restaurant,American Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Southern / Soul Food Restaurant,Seafood Restaurant,Cuban Restaurant,French Restaurant,Empanada Restaurant
40,Olinville,Fast Food Restaurant,Caribbean Restaurant,Chinese Restaurant,Vegetarian / Vegan Restaurant,American Restaurant,Mexican Restaurant,Empanada Restaurant,Indian Restaurant,Greek Restaurant,French Restaurant
44,Edenwald,Fast Food Restaurant,Caribbean Restaurant,Chinese Restaurant,Vietnamese Restaurant,Italian Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Comfort Food Restaurant,Cuban Restaurant
45,Claremont Village,Caribbean Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Italian Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant
50,Allerton,Caribbean Restaurant,Chinese Restaurant,Fast Food Restaurant,Mexican Restaurant,American Restaurant,Vietnamese Restaurant,Empanada Restaurant,Indian Restaurant,Greek Restaurant,French Restaurant


###Cluster Three

In [31]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 2, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Co-op City,Fast Food Restaurant,Caribbean Restaurant,Mexican Restaurant,Restaurant,American Restaurant,Spanish Restaurant,Chinese Restaurant,Seafood Restaurant,Eastern European Restaurant,French Restaurant
3,Fieldston,Mexican Restaurant,Restaurant,Sushi Restaurant,Fast Food Restaurant,Chinese Restaurant,Greek Restaurant,Tapas Restaurant,Asian Restaurant,Caribbean Restaurant,Indian Restaurant
4,Riverdale,Japanese Restaurant,Mexican Restaurant,Tapas Restaurant,Italian Restaurant,Restaurant,Thai Restaurant,Greek Restaurant,Arepa Restaurant,Asian Restaurant,Caribbean Restaurant
5,Kingsbridge,Mexican Restaurant,Japanese Restaurant,Spanish Restaurant,Seafood Restaurant,Italian Restaurant,Latin American Restaurant,Restaurant,Caribbean Restaurant,Tapas Restaurant,Thai Restaurant
7,Norwood,Mexican Restaurant,Spanish Restaurant,Fast Food Restaurant,Caribbean Restaurant,Chinese Restaurant,Restaurant,Latin American Restaurant,Eastern European Restaurant,French Restaurant,Empanada Restaurant
9,Baychester,Italian Restaurant,Spanish Restaurant,Chinese Restaurant,Mexican Restaurant,American Restaurant,Caribbean Restaurant,Seafood Restaurant,Fast Food Restaurant,Empanada Restaurant,Greek Restaurant
12,Bedford Park,Mexican Restaurant,Spanish Restaurant,Chinese Restaurant,Italian Restaurant,Caribbean Restaurant,Fast Food Restaurant,New American Restaurant,Empanada Restaurant,Greek Restaurant,French Restaurant
13,University Heights,Spanish Restaurant,Latin American Restaurant,Fast Food Restaurant,African Restaurant,Chinese Restaurant,Italian Restaurant,Mexican Restaurant,Restaurant,Comfort Food Restaurant,Indian Restaurant
14,Morris Heights,Spanish Restaurant,Fast Food Restaurant,Latin American Restaurant,Asian Restaurant,Mexican Restaurant,Chinese Restaurant,Israeli Restaurant,Vietnamese Restaurant,Empanada Restaurant,Greek Restaurant
18,High Bridge,Theme Restaurant,Mexican Restaurant,Caribbean Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Greek Restaurant,French Restaurant,Fast Food Restaurant,Empanada Restaurant,Comfort Food Restaurant


###Cluster Four

In [32]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 3, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,East Tremont,Fast Food Restaurant,Puerto Rican Restaurant,American Restaurant,Chinese Restaurant,Restaurant,Vietnamese Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant,Eastern European Restaurant
17,West Farms,Fast Food Restaurant,Puerto Rican Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Indian Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant,Cuban Restaurant,Italian Restaurant
22,Longwood,Fast Food Restaurant,Mexican Restaurant,Latin American Restaurant,Chinese Restaurant,Spanish Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant
24,Morrisania,Fast Food Restaurant,Latin American Restaurant,Mexican Restaurant,Italian Restaurant,Caribbean Restaurant,Spanish Restaurant,Chinese Restaurant,Empanada Restaurant,Greek Restaurant,French Restaurant
25,Soundview,Fast Food Restaurant,Chinese Restaurant,American Restaurant,Southern / Soul Food Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Indian Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant
47,Mount Eden,Fast Food Restaurant,Spanish Restaurant,Latin American Restaurant,Chinese Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant,Comfort Food Restaurant
48,Mount Hope,Fast Food Restaurant,Latin American Restaurant,Italian Restaurant,Spanish Restaurant,Chinese Restaurant,Eastern European Restaurant,Indian Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant


#####Let's see how many Neighborhood by Cluster

In [33]:
bronx_merged.groupby('Cluster Labels').count()

,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,
0,15,15,15,15,15,15,15,15,15,15,15,15,15,15
1,7,7,7,7,7,7,7,7,7,7,7,7,7,7
2,22,22,22,22,22,22,22,22,22,22,22,22,22,22
3,7,7,7,7,7,7,7,7,7,7,7,7,7,7


#####We can figure out **Cluster three** it's the most densed based on Restarant count, that represent our competitor. those neighborhood are not the best site selection site just taking into consideration venues similarity.

**Cluster four** seems to be the best Neighborhood locations to be considered to open a new restaurant.


#####Analyze cluster four deeper to determine the best neighborhood

In [34]:
bronx_venues_rest.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Caribbean Restaurant
2,Wakefield,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Caribbean Restaurant
3,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant
10,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant,40.898083,-73.850259,Caribbean Restaurant
19,Wakefield,40.894705,-73.847201,McDonald's,40.902645,-73.849485,Fast Food Restaurant


In [35]:
#Dataframe for Cluster four
bronx_merged_cluster4 = bronx_merged.join(bronx_venues_rest.set_index('Neighborhood'), on='Neighborhood',how='inner')

bronx_merged_cluster4.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bronx,Wakefield,40.894705,-73.847201,1,Caribbean Restaurant,Fast Food Restaurant,Chinese Restaurant,Spanish Restaurant,Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Caribbean Restaurant
0,Bronx,Wakefield,40.894705,-73.847201,1,Caribbean Restaurant,Fast Food Restaurant,Chinese Restaurant,Spanish Restaurant,Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Caribbean Restaurant
0,Bronx,Wakefield,40.894705,-73.847201,1,Caribbean Restaurant,Fast Food Restaurant,Chinese Restaurant,Spanish Restaurant,Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant
0,Bronx,Wakefield,40.894705,-73.847201,1,Caribbean Restaurant,Fast Food Restaurant,Chinese Restaurant,Spanish Restaurant,Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant,40.898083,-73.850259,Caribbean Restaurant
0,Bronx,Wakefield,40.894705,-73.847201,1,Caribbean Restaurant,Fast Food Restaurant,Chinese Restaurant,Spanish Restaurant,Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant,40.894705,-73.847201,McDonald's,40.902645,-73.849485,Fast Food Restaurant


#####Let's Slice Cluster 4 in order to plot the map

In [36]:
bronx_venues_cluster4 =bronx_merged_cluster4[bronx_merged_cluster4['Cluster Labels'] == 3]
bronx_venues_cluster4.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
16,Bronx,East Tremont,40.842696,-73.887356,3,Fast Food Restaurant,Puerto Rican Restaurant,American Restaurant,Chinese Restaurant,Restaurant,Vietnamese Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant,Eastern European Restaurant,40.842696,-73.887356,El Nuevo Bohio,40.843956,-73.888025,Puerto Rican Restaurant
16,Bronx,East Tremont,40.842696,-73.887356,3,Fast Food Restaurant,Puerto Rican Restaurant,American Restaurant,Chinese Restaurant,Restaurant,Vietnamese Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant,Eastern European Restaurant,40.842696,-73.887356,Wendy’s,40.842810,-73.897865,Fast Food Restaurant
16,Bronx,East Tremont,40.842696,-73.887356,3,Fast Food Restaurant,Puerto Rican Restaurant,American Restaurant,Chinese Restaurant,Restaurant,Vietnamese Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant,Eastern European Restaurant,40.842696,-73.887356,Taco Bell,40.837344,-73.884434,Fast Food Restaurant
16,Bronx,East Tremont,40.842696,-73.887356,3,Fast Food Restaurant,Puerto Rican Restaurant,American Restaurant,Chinese Restaurant,Restaurant,Vietnamese Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant,Eastern European Restaurant,40.842696,-73.887356,El Valle Restaurant,40.844964,-73.889647,Restaurant
16,Bronx,East Tremont,40.842696,-73.887356,3,Fast Food Restaurant,Puerto Rican Restaurant,American Restaurant,Chinese Restaurant,Restaurant,Vietnamese Restaurant,Greek Restaurant,French Restaurant,Empanada Restaurant,Eastern European Restaurant,40.842696,-73.887356,The Dancing Crane Cafe,40.849698,-73.880404,American Restaurant


In [37]:
bronx_venues_cluster4.shape

(67, 21)

#####Let's visualize cluster four Location.

In [39]:
# create map of Bronx using latitude and longitude values
map_bronx_venues_restaurant_cluster4 = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label, Cluster in zip(bronx_venues_cluster4['Venue Latitude'], bronx_venues_cluster4['Venue Longitude'], bronx_venues_cluster4['Venue'], bronx_venues_cluster4['Cluster Labels']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color = 'blue' ,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_bronx_venues_restaurant_cluster4)  
    
map_bronx_venues_restaurant_cluster4

###Results and Discussion
Our analysis shows that although there is a great number of restaurants in Bronx 596 after slicing first data location, we find low restaurant density in the north region. Highest concentration of restaurants was detected south and and west from center, so we focused our attention to areas south, corresponding to boroughs **East Tremon, Mount Hope, Mount Eden, Morrisania, Longwood**. Other boroughs identified as potentially interesting was Soundview.
We first created Neighborhood clustering and segmenting them based on restaurants categories for exploring deeper our zone of interest. We divided in four groups of 10 most common type of restaurant for making those with most Latin and Caribbean restaurants the zone of our interest.
Result of all this is four cluster being cluster number one containing largest number of 1srt Most common venue of Caribbean restaurant, and cluster number four having the lowest 1srt Most common venue of Caribbean restaurant. So this cluster has the largest number of potential new restaurant locations based on number of 1srt Most common venue of Caribbean restaurant.
Recommended zones should only be considered as staring point for more detailed analysis, like Latin population density. And what competitors offers.


###Conclusion
Purpose of this project was to identify **Bronx borough low number restaurants** particularly Caribbean and Latin restaurants in order to help stakeholders in narrowing down the search for optimal location for a new Latin restaurant. By calculating restaurant density distribution from Foursquare data we have first identified general boroughs that justify further analysis (East Tremon, Mount Hope, Mount Eden, Morrisania, Longwood), and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.
